In [1]:
%config Completer.use_jedi = False

import configparser
config = configparser.ConfigParser()
config.read('configBath2p.txt')

source_folder = config.get('Configuration', 'source_folder')
time_step = int(config.get('Configuration', 'time_step'))
window_size = int(config.get('Configuration', 'window_size'))
data_folder = config.get('Configuration', 'data_folder')
ini_date = config.get('Configuration', 'ini_date')
end_date = config.get('Configuration', 'end_date')

users=config.get('Configuration', 'users')
users = [(item.strip()) for item in users.split(',')]

room=config.get('Configuration', 'room')
rooms=[room]

w = int(config.get(room, 'w'))
h = int(config.get(room, 'h'))




In [2]:
import numpy as np
from pandas import read_csv    
import pandas as pd
import time

from datetime import datetime

off_zone=60*60*2

def day_time(ti):
    return int((int)((ti+off_zone)/(60*60*24)))


def time2str(tt):
    return datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S")

tN = (int)(datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (end_date,"End date is", tN, "day:",day_time(tN), time2str(tN))


print(ini_date)
t0 = (int)(datetime.strptime(ini_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (ini_date,"Init date is", t0, "day:",day_time(t0), time2str(t0))




def day_time0(ti):
    return day_time(ti)-day_time(t0)+1

def relT(ti):
    return (int)((ti-t0)/time_step)

#day=list(range(day_time(t0),day_time(tN)+1))[0]
day=19798
print(day)

2024-03-16 20:59:48 End date is 1710619188 day: 19798 2024-03-16 20:59:48
2024-03-16 20:50:46
2024-03-16 20:50:46 Init date is 1710618646 day: 19798 2024-03-16 20:50:46
19798


In [3]:
def getLoc(room,s,off):
    print("S:",s)
    xys=config.get(room, s)
    xys = [(item.strip()) for item in xys.split(',')]
    xys = [list(map(int, item.split('x'))) for item in xys]
    for xy in xys:
        print("\t",xy)
        xy[0]=xy[0]+off[0]
        xy[1]=xy[1]+off[1]
        print("\t+",xy)
    return xys

loc_room_sensor={}
fp_sensors={}
for room in rooms:
    print(room)
    loc_room_sensor[room]={}
    fp_sensors[room] = config.get(room, 'maps')
    fp_sensors[room]  = [(item.strip()) for item in fp_sensors[room].split(',')]

    offW = int(config.get(room, 'offW'))
    offH = int(config.get(room, 'offH'))

    for fp in fp_sensors[room]:
        print(fp)
        loc_room_sensor[room][fp]=getLoc(room,fp,(offH,offW))
        print(loc_room_sensor[room][fp])
print(loc_room_sensor)
print(fp_sensors)

bathroom
b1
S: b1
	 [0, 0]
	+ [0, 0]
[[0, 0]]
b2
S: b2
	 [1, 0]
	+ [1, 0]
[[1, 0]]
b3
S: b3
	 [2, 0]
	+ [2, 0]
[[2, 0]]
b4
S: b4
	 [3, 0]
	+ [3, 0]
[[3, 0]]
b5
S: b5
	 [4, 0]
	+ [4, 0]
[[4, 0]]
b6
S: b6
	 [5, 0]
	+ [5, 0]
[[5, 0]]
b7
S: b7
	 [6, 0]
	+ [6, 0]
[[6, 0]]
b8
S: b8
	 [0, 1]
	+ [0, 1]
[[0, 1]]
b9
S: b9
	 [1, 1]
	+ [1, 1]
[[1, 1]]
b10
S: b10
	 [2, 1]
	+ [2, 1]
[[2, 1]]
b11
S: b11
	 [3, 1]
	+ [3, 1]
[[3, 1]]
b12
S: b12
	 [4, 1]
	+ [4, 1]
[[4, 1]]
b13
S: b13
	 [5, 1]
	+ [5, 1]
[[5, 1]]
b14
S: b14
	 [6, 1]
	+ [6, 1]
[[6, 1]]
b15
S: b15
	 [0, 2]
	+ [0, 2]
[[0, 2]]
b16
S: b16
	 [1, 2]
	+ [1, 2]
[[1, 2]]
b17
S: b17
	 [2, 2]
	+ [2, 2]
[[2, 2]]
b18
S: b18
	 [3, 2]
	+ [3, 2]
[[3, 2]]
b19
S: b19
	 [4, 2]
	+ [4, 2]
[[4, 2]]
b20
S: b20
	 [5, 2]
	+ [5, 2]
[[5, 2]]
b21
S: b21
	 [6, 2]
	+ [6, 2]
[[6, 2]]
{'bathroom': {'b1': [[0, 0]], 'b2': [[1, 0]], 'b3': [[2, 0]], 'b4': [[3, 0]], 'b5': [[4, 0]], 'b6': [[5, 0]], 'b7': [[6, 0]], 'b8': [[0, 1]], 'b9': [[1, 1]], 'b10': [[2, 1]], 'b11': [[3, 1]]

In [4]:
import os

df_total_map = {}
for room in rooms:
    df_total_map[room]={}
    print(os.listdir(os.path.join(source_folder)))
    for fp in fp_sensors[room]:
        df_total_map[room][fp]=None
    for scene in os.listdir(os.path.join(source_folder)):
        for fp in fp_sensors[room]:
            print("scene",scene)
            print(os.path.join(source_folder,scene)+"/binary/"+str(day)+"."+fp+".tsv")
            try:
                series=read_csv(os.path.join(source_folder,scene)+"/binary/"+str(day)+"."+fp+".tsv", sep="\t",parse_dates=True, header=None, names=["time","date","state"])
                print(day,fp,series.size)
            except:
                print("Not data")
                continue
            if(df_total_map[room][fp] is None):
                df_total_map[room][fp]=series
            else:
                df_total_map[room][fp] = df_total_map[room][fp].append(series, ignore_index=True)


for room in rooms:
    for fp in fp_sensors[room]:
        if(df_total_map[room][fp] is not None):
            df_total_map[room][fp] = df_total_map[room][fp].sort_values(by='time', ascending=True)
        print(df_total_map[room][fp])


['Scene1', 'Scene2', 'Scene3']
scene Scene1
./data/bathroom/2-person-labelled\Scene1/binary/19798.b1.tsv
19798 b1 9
scene Scene1
./data/bathroom/2-person-labelled\Scene1/binary/19798.b2.tsv
19798 b2 15
scene Scene1
./data/bathroom/2-person-labelled\Scene1/binary/19798.b3.tsv
19798 b3 27
scene Scene1
./data/bathroom/2-person-labelled\Scene1/binary/19798.b4.tsv
19798 b4 27
scene Scene1
./data/bathroom/2-person-labelled\Scene1/binary/19798.b5.tsv
19798 b5 15
scene Scene1
./data/bathroom/2-person-labelled\Scene1/binary/19798.b6.tsv
19798 b6 9
scene Scene1
./data/bathroom/2-person-labelled\Scene1/binary/19798.b7.tsv
19798 b7 9
scene Scene1
./data/bathroom/2-person-labelled\Scene1/binary/19798.b8.tsv
19798 b8 9
scene Scene1
./data/bathroom/2-person-labelled\Scene1/binary/19798.b9.tsv
19798 b9 15
scene Scene1
./data/bathroom/2-person-labelled\Scene1/binary/19798.b10.tsv
19798 b10 30
scene Scene1
./data/bathroom/2-person-labelled\Scene1/binary/19798.b11.tsv
19798 b11 21
scene Scene1
./data/bat

C:\Users\Usuario\AppData\Local\Temp\ipykernel_39836\654460643.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[room][fp] = df_total_map[room][fp].append(series, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_39836\654460643.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[room][fp] = df_total_map[room][fp].append(series, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_39836\654460643.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[room][fp] = df_total_map[room][fp].append(series, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_39836\654460643.py:22: FutureWarning: The frame.append method is deprecated and will be removed 

19798 b6 27
scene Scene2
./data/bathroom/2-person-labelled\Scene2/binary/19798.b7.tsv
19798 b7 6
scene Scene2
./data/bathroom/2-person-labelled\Scene2/binary/19798.b8.tsv
19798 b8 9
scene Scene2
./data/bathroom/2-person-labelled\Scene2/binary/19798.b9.tsv
19798 b9 6
scene Scene2
./data/bathroom/2-person-labelled\Scene2/binary/19798.b10.tsv
19798 b10 24
scene Scene2
./data/bathroom/2-person-labelled\Scene2/binary/19798.b11.tsv
19798 b11 18
scene Scene2
./data/bathroom/2-person-labelled\Scene2/binary/19798.b12.tsv
19798 b12 30
scene Scene2
./data/bathroom/2-person-labelled\Scene2/binary/19798.b13.tsv
19798 b13 24
scene Scene2
./data/bathroom/2-person-labelled\Scene2/binary/19798.b14.tsv
19798 b14 12
scene Scene2
./data/bathroom/2-person-labelled\Scene2/binary/19798.b15.tsv
19798 b15 3
scene Scene2
./data/bathroom/2-person-labelled\Scene2/binary/19798.b16.tsv
19798 b16 6
scene Scene2
./data/bathroom/2-person-labelled\Scene2/binary/19798.b17.tsv
19798 b17 9
scene Scene2
./data/bathroom/2-p

C:\Users\Usuario\AppData\Local\Temp\ipykernel_39836\654460643.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[room][fp] = df_total_map[room][fp].append(series, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_39836\654460643.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[room][fp] = df_total_map[room][fp].append(series, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_39836\654460643.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[room][fp] = df_total_map[room][fp].append(series, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_39836\654460643.py:22: FutureWarning: The frame.append method is deprecated and will be removed 

19798 b3 39
scene Scene3
./data/bathroom/2-person-labelled\Scene3/binary/19798.b4.tsv
19798 b4 30
scene Scene3
./data/bathroom/2-person-labelled\Scene3/binary/19798.b5.tsv
19798 b5 42
scene Scene3
./data/bathroom/2-person-labelled\Scene3/binary/19798.b6.tsv
19798 b6 24
scene Scene3
./data/bathroom/2-person-labelled\Scene3/binary/19798.b7.tsv
19798 b7 6
scene Scene3
./data/bathroom/2-person-labelled\Scene3/binary/19798.b8.tsv
19798 b8 6
scene Scene3
./data/bathroom/2-person-labelled\Scene3/binary/19798.b9.tsv
19798 b9 24
scene Scene3
./data/bathroom/2-person-labelled\Scene3/binary/19798.b10.tsv
19798 b10 27
scene Scene3
./data/bathroom/2-person-labelled\Scene3/binary/19798.b11.tsv
19798 b11 30
scene Scene3
./data/bathroom/2-person-labelled\Scene3/binary/19798.b12.tsv
19798 b12 27
scene Scene3
./data/bathroom/2-person-labelled\Scene3/binary/19798.b13.tsv
19798 b13 18
scene Scene3
./data/bathroom/2-person-labelled\Scene3/binary/19798.b14.tsv
19798 b14 18
scene Scene3
./data/bathroom/2-per

C:\Users\Usuario\AppData\Local\Temp\ipykernel_39836\654460643.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[room][fp] = df_total_map[room][fp].append(series, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_39836\654460643.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[room][fp] = df_total_map[room][fp].append(series, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_39836\654460643.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[room][fp] = df_total_map[room][fp].append(series, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_39836\654460643.py:22: FutureWarning: The frame.append method is deprecated and will be removed 

In [5]:
import cv2
import os

def getStateTime(df_total,tt):

    df_intervalo1 = df_total[(df_total['time'] >= (tt))]
    df_intervalo2 = df_total[(df_total['time'] <= (tt))]
    #print("tt",tt,time2str(tt))
    #print("df_intervalo1",df_intervalo1, len(df_intervalo1))
    #print("df_intervalo2",df_intervalo2, time2str(tt),tt,time2str(df_intervalo2.iloc[-1]['time']),df_intervalo2.iloc[-1]['time'])
    if(len(df_intervalo2)==0):
        return 100,0
    if(df_intervalo2.iloc[-1]['state']=='on'):
        return tt-df_intervalo2.iloc[-1]['time'],1
    if(df_intervalo2.iloc[-1]['state']=='off'):
        return tt-df_intervalo2.iloc[-1]['time'],-1
    return 100,0
    

os.makedirs(data_folder+"B/DAY_"+str(day)+"/", exist_ok=True)
print(t0,tN,time_step)
for tt in range(t0,tN,time_step):
    print(tt,day_time(tt),time2str(tt))
    #print(tt-off_zone,day_time(tt-off_zone),time2str(tt-off_zone))
    #if(day_time(tt) != day):
    #    continue


    img0=np.zeros((h, w))
    v0={}
    min_t={}
    for room in rooms:
        for fp in fp_sensors[room]:
            if not fp in v0:
                v0[fp]=0
                min_t[fp]=100
                
        
        
    for room in rooms:
        for fp in fp_sensors[room]:
            df_total=df_total_map[room][fp]  
            inct,v=getStateTime(df_total,tt)
            #v0[fp]=v0[fp]+v
            #19:35:17
            #if(fp=="s2"):
            #    print("\t",room,fp,inct,v)
            if(inct<min_t[fp] and inct<15):
                min_t[fp]=inct
                v0[fp]=v
                
    for room in rooms:
        for fp in fp_sensors[room]:
            for xy in loc_room_sensor[room][fp]:
                #if(fp=="s2"):
                #    print("\t",v0[fp])
                img0[xy[0]][xy[1]]=img0[xy[0]][xy[1]]+int(v0[fp]*255)
        
            #df_intervalo = df_total[(df_total['time'] >= (tt-window_size/2)) & (df_total['time'] <= (tt)) & (df_total['state']=='on')]
            #print("DF",df_intervalo)            
            #for index, row in df_total.iterrows():
            #    dif=tt-row['time']+1
            #    print("@TIME",tt,fp,row, row['time'], "dif:", dif)
            #    for xy in loc_sensor[fp]:
            #        img0[xy[0]][xy[1]]=img0[xy[0]][xy[1]]+int(255/dif)
            #for ti in range(int(tt-1),tt+1):
                #print(ti)
                #inct,v=getStateTime(df_total,ti)*0.5
                #dif=tt-ti+1
                #print("@TIMA ",tt,v, "difA:", dif)
                #print(fp,"ti")
                #for xy in loc_room_sensor[room][fp]:
                    #print(fp,xy, "->",v)
                #    img0[xy[0]][xy[1]]=img0[xy[0]][xy[1]]+int(v*255/dif)

            #for ti in range(tt+1, int(tt+1+window_size/2)):
            #    v=getStateTime(df_total,ti)*0.25
            #    dif=ti-tt+1
                #print("@TIMB ",tt,v, "difB:", dif)
            #    for xy in loc_room_sensor[room][fp]:
            #        img0[xy[0]][xy[1]]=img0[xy[0]][xy[1]]+int(v*255/dif)

    img0[img0 < 0] = 0
    img0[img0 > 255] = 255
    cv2.imwrite(data_folder+"B/DAY_"+str(day)+"/FP."+str(tt)+".png", img0) 

1710618646 1710619188 1
1710618646 19798 2024-03-16 20:50:46
1710618647 19798 2024-03-16 20:50:47
1710618648 19798 2024-03-16 20:50:48
1710618649 19798 2024-03-16 20:50:49
1710618650 19798 2024-03-16 20:50:50
1710618651 19798 2024-03-16 20:50:51
1710618652 19798 2024-03-16 20:50:52
1710618653 19798 2024-03-16 20:50:53
1710618654 19798 2024-03-16 20:50:54
1710618655 19798 2024-03-16 20:50:55
1710618656 19798 2024-03-16 20:50:56
1710618657 19798 2024-03-16 20:50:57
1710618658 19798 2024-03-16 20:50:58
1710618659 19798 2024-03-16 20:50:59
1710618660 19798 2024-03-16 20:51:00
1710618661 19798 2024-03-16 20:51:01
1710618662 19798 2024-03-16 20:51:02
1710618663 19798 2024-03-16 20:51:03
1710618664 19798 2024-03-16 20:51:04
1710618665 19798 2024-03-16 20:51:05
1710618666 19798 2024-03-16 20:51:06
1710618667 19798 2024-03-16 20:51:07
1710618668 19798 2024-03-16 20:51:08
1710618669 19798 2024-03-16 20:51:09
1710618670 19798 2024-03-16 20:51:10
1710618671 19798 2024-03-16 20:51:11
1710618672 197

1710618875 19798 2024-03-16 20:54:35
1710618876 19798 2024-03-16 20:54:36
1710618877 19798 2024-03-16 20:54:37
1710618878 19798 2024-03-16 20:54:38
1710618879 19798 2024-03-16 20:54:39
1710618880 19798 2024-03-16 20:54:40
1710618881 19798 2024-03-16 20:54:41
1710618882 19798 2024-03-16 20:54:42
1710618883 19798 2024-03-16 20:54:43
1710618884 19798 2024-03-16 20:54:44
1710618885 19798 2024-03-16 20:54:45
1710618886 19798 2024-03-16 20:54:46
1710618887 19798 2024-03-16 20:54:47
1710618888 19798 2024-03-16 20:54:48
1710618889 19798 2024-03-16 20:54:49
1710618890 19798 2024-03-16 20:54:50
1710618891 19798 2024-03-16 20:54:51
1710618892 19798 2024-03-16 20:54:52
1710618893 19798 2024-03-16 20:54:53
1710618894 19798 2024-03-16 20:54:54
1710618895 19798 2024-03-16 20:54:55
1710618896 19798 2024-03-16 20:54:56
1710618897 19798 2024-03-16 20:54:57
1710618898 19798 2024-03-16 20:54:58
1710618899 19798 2024-03-16 20:54:59
1710618900 19798 2024-03-16 20:55:00
1710618901 19798 2024-03-16 20:55:01
1

1710619098 19798 2024-03-16 20:58:18
1710619099 19798 2024-03-16 20:58:19
1710619100 19798 2024-03-16 20:58:20
1710619101 19798 2024-03-16 20:58:21
1710619102 19798 2024-03-16 20:58:22
1710619103 19798 2024-03-16 20:58:23
1710619104 19798 2024-03-16 20:58:24
1710619105 19798 2024-03-16 20:58:25
1710619106 19798 2024-03-16 20:58:26
1710619107 19798 2024-03-16 20:58:27
1710619108 19798 2024-03-16 20:58:28
1710619109 19798 2024-03-16 20:58:29
1710619110 19798 2024-03-16 20:58:30
1710619111 19798 2024-03-16 20:58:31
1710619112 19798 2024-03-16 20:58:32
1710619113 19798 2024-03-16 20:58:33
1710619114 19798 2024-03-16 20:58:34
1710619115 19798 2024-03-16 20:58:35
1710619116 19798 2024-03-16 20:58:36
1710619117 19798 2024-03-16 20:58:37
1710619118 19798 2024-03-16 20:58:38
1710619119 19798 2024-03-16 20:58:39
1710619120 19798 2024-03-16 20:58:40
1710619121 19798 2024-03-16 20:58:41
1710619122 19798 2024-03-16 20:58:42
1710619123 19798 2024-03-16 20:58:43
1710619124 19798 2024-03-16 20:58:44
1